In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from random import seed, randint
import json
from scipy.io import loadmat, savemat
from scipy.spatial.distance import euclidean, mahalanobis
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.neighbors import NearestCentroid

In [ ]:
mcl = {'EO': loadmat('MCL.mat')['EO'].reshape(109, 6, 64, 1), 'EC': loadmat('MCL.mat')['EC'].reshape(109, 6, 64, 1)}
kfd = {'EO': loadmat('KFD.mat')['EO'].reshape(109, 6, 64, 1), 'EC': loadmat('KFD.mat')['EC'].reshape(109, 6, 64, 1)}
psd = {'EO': loadmat('PSD.mat')['EO'], 'EC': loadmat('PSD.mat')['EC']}
ap = {'EO': loadmat('AP.mat')['EO'], 'EC': loadmat('AP.mat')['EC']}
plv = {'EO': loadmat('PLV.mat')['EO'], 'EC': loadmat('PLV.mat')['EC']}
coh = {'EO': loadmat('COH.mat')['EO'], 'EC': loadmat('COH.mat')['EC']}
ar = {'EO': loadmat('AR.mat')['EO'], 'EC': loadmat('AR.mat')['EC']}

In [ ]:
class Euclidean(BaseEstimator, ClassifierMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y):
        self.y = np.array(list(set(y)))
        self.X = np.zeros((len(self.y), X.shape[1]))
        for i in range(len(self.y)):
            self.X[i] = X[y==self.y[i]].mean(axis=0)
        
        return self
    
    def predict(self, X):
        p = np.ones(X.shape[0], dtype='int32')*self.y[0]
        for i, x in enumerate(X):
            s = euclidean(x, self.X[0])
            for j in range(1, len(self.y)):
                d = euclidean(x, self.X[j])
                if d < s:
                    s = d
                    p[i] = self.y[j]
        return p
    
    def predict_proba(self, X):
        p = np.zeros((X.shape[0], self.X.shape[0]))
        for i, x in enumerate(X):
            for j in range(len(self.y)):
                p[i, j] = euclidean(x, self.X[j])
        return p

In [ ]:
class Mahalanobis(BaseEstimator, ClassifierMixin):
    def __init__(self, VI=None):
        self.VI = VI
    
    def fit(self, X, y):
        self.y = np.array(list(set(y)))
        self.X = np.zeros((len(self.y), X.shape[1]))
        for i in range(len(self.y)):
            self.X[i] = X[y==self.y[i]].mean(axis=0)
        if self.VI is None:
            self.VI = np.linalg.inv(np.cov(X.transpose()))
        
        return self
    
    def predict(self, X):
        p = np.ones(X.shape[0], dtype='int32')*self.y[0]
        for i, x in enumerate(X):
            s = mahalanobis(x, self.X[0], self.VI)
            for j in range(1, len(self.y)):
                d = mahalanobis(x, self.X[j], self.VI)
                if d < s:
                    s = d
                    p[i] = self.y[j]
        return p
    
    def predict_proba(self, X):
        p = np.zeros((X.shape[0], self.X.shape[0]))
        for i, x in enumerate(X):
            for j in range(len(self.y)):
                p[i, j] = mahalanobis(x, self.X[j], self.VI)
        return p

# Identification

In [ ]:
# Table 1 & 2

features = {'MCL': mcl, 'KFD': kfd, 'PSD': psd, 'AP': ap, 'PLV': plv, 'COH': coh, 'AR': ar}
protocols = ['EO', 'EC']
accuracy_scores = {'MCL': {'EO': None, 'EC': None}, 'KFD': {'EO': None, 'EC': None},
                   'PSD': {'EO': None, 'EC': None}, 'AP':  {'EO': None, 'EC': None},
                   'PLV': {'EO': None, 'EC': None}, 'COH': {'EO': None, 'EC': None},
                   'AR':  {'EO': None, 'EC': None}}
n_components = accuracy_scores.copy()
y_train = np.array([i for i in range(109) for j in range(5)])
y_test = np.array([i for i in range(109)])

for feature in features.keys():
    if feature in {'PLV', 'COH', 'AR'}:
        n_comp = 0.99
    else:
        n_comp = None
    for protocol in protocols:
        accuracy_scores_ = [0]*6
        n_components_ = [0]*6
        for i in range(6):
            
            train_set = list(range(6))
            test_set = [train_set.pop(i)]
            
            x_train = features[feature][protocol][:, train_set, :, :].reshape(109*5, -1)
            x_test = features[feature][protocol][:, test_set, :, :].reshape(109*1, -1)
            
            pca = PCA(n_components=n_comp, whiten=True)
            clf = NearestCentroid('euclidean')
            model = Pipeline([('pca', pca), ('clf', clf)])
            model.fit(x_train, y_train)
            
            accuracy_scores_[i] = accuracy_score(y_test, model.predict(x_test))*100
            n_components_[i] = model.named_steps['pca'].n_components_
        
        accuracy_scores[feature][protocol] = accuracy_scores_
        n_components[feature][protocol] = n_components_
        print(feature+'-'+protocol+':',
              np.mean(accuracy_scores_).round(1), '\N{PLUS-MINUS SIGN}', np.std(accuracy_scores_).round(1), '%',
              '\t', int(np.mean(n_components_).round()), 'p.c.')

In [ ]:
# Table 2

print('Average Number of Components\n')
for feature in {'PLV', 'COH', 'AR'}:
    print(feature+':', int(np.mean([n_components[feature]['EO'], n_components[feature]['EC']]).round()))

# Authentication

In [ ]:
y_true = np.array([i==j for i in range(109) for j in range(109)]*6)
def cross_validate(data, N, n_comp=None):
    y_train = np.array([i for i in range(N) for j in range(5)])
    y_score = []
    
    for i in range(6):
        
        train_set = list(range(6))
        test_set = [train_set.pop(i)]
        
        x_train = data[:N, train_set, :, :].reshape([N*5, -1])
        x_test = data[:N, test_set, :, :].reshape([N*1, -1])
        if N < 109:
            x_imposter = data[N:, :, :, :].reshape([(109-N)*6, -1])
        
        pca = PCA(n_components=n_comp, whiten=True)
        clf = Euclidean()
        model = Pipeline([('pca', pca), ('clf', clf)])
        model.fit(x_train, y_train)
        
        y_score.extend(model.predict_proba(x_test).ravel())
    
    return np.array(y_score)

In [ ]:
protocols = ['EO', 'EC']
features = {'MCL': mcl, 'KFD': kfd, 'PSD': psd, 'AP': ap, 'PLV': plv, 'COH': coh, 'AR': ar}
y_score = dict()
for feature in features.keys():
    y_score[feature] = {'EO': None, 'EC': None}

for feature in features.keys():
    if feature in ['PLV', 'COH', 'AR']:
        n_comp = 0.99
    else:
        n_comp = None
    for protocol in protocols:
        y_score[feature][protocol] = cross_validate(features[feature][protocol], 109, n_comp)
    print(feature+' done.')

In [ ]:
# since distance is a dissimilarity-based measure negatives are positives and vice versa
fpr = dict()
fnr = dict()
eer = dict()
for feature in features.keys():
    fpr[feature] = {'EO': None, 'EC': None}
    fnr[feature] = {'EO': None, 'EC': None}
    eer[feature] = {'EO': None, 'EC': None}

for feature in features.keys():
    for protocol in protocols:
        fpr_tmp, tpr_tmp, _ = roc_curve(y_true, -y_score[feature][protocol])
        fnr_tmp = 1-tpr_tmp
        fpr[feature][protocol] = fpr_tmp
        fnr[feature][protocol] = fnr_tmp

In [ ]:
# Fig. 3

protocol_titles = ['Eyes Open', 'Eyes Close']
line_format = {'MCL': {'EO': 'b', 'EC': 'b'}, 'KFD': {'EO': 'r', 'EC': 'r'},
              'PSD': {'EO': 'g', 'EC': 'g'},  'AP': {'EO': 'y', 'EC': 'y'},
              'PLV': {'EO': 'c', 'EC': 'c'}, 'COH': {'EO': 'm', 'EC': 'm'},
               'AR': {'EO': 'k', 'EC': 'k'}}

for i, protocol in enumerate(protocols):
    
    legend = []
    
    plt.subplot(1, 2, i+1)
    for feature in features.keys():
        if feature != 'MCL':
            lw = 0.75
        else:
            lw = 3
        plt.plot(fpr[feature][protocol], fnr[feature][protocol], line_format[feature][protocol], linewidth=lw)
        legend.append(feature)
    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.text(0.30, 0.37, 'EER Line', fontsize=12, weight='semibold', color='black')
    
    plt.xticks(fontsize=12), plt.yticks(fontsize=12)
    plt.xlabel('False Acceptance Rate', fontsize=16), plt.ylabel('False Rejection Rate', fontsize=16)
    plt.title(protocol_titles[i], weight='normal', fontsize=18)
    plt.legend(legend, fontsize=12, loc='right')
    
    plt.grid(ls=':')
    plt.axis('square')
    plt.xlim([0, 0.4]), plt.ylim([0, 0.4])
    
plt.gcf().set_size_inches(15, 15)

for feature in features.keys():
    for protocol in protocols:
        eer_idx = np.argmin(np.abs(fpr[feature][protocol]-fnr[feature][protocol]))
        eer[feature][protocol] = np.mean([fpr[feature][protocol][eer_idx], fnr[feature][protocol][eer_idx]]).round(4)

In [ ]:
# Table 1

eer

# Channel Ranks

In [ ]:
def cross_validate(model, chan_set, protocol):
    protocol = 'EO'
    acc = 0
    for i in range(6):
        train_set = list(range(6))
        test_set = [train_set.pop(i)]
        x_train = mcl[protocol][:, train_set, :][:, :, chan_set].reshape(109*5, -1)
        x_test = mcl[protocol][:, test_set, :][:, :, chan_set].reshape(109*1, -1)
        model.fit(x_train, y_train)
        acc += accuracy_score(y_test, model.predict(x_test))/6*100
    return acc

In [ ]:
protocols = ['EO', 'EC']
ranked_chans = {'EO': [0]*64, 'EC': [0]*64}

for protocol in protocols:
    accuracy_scores = [0]*64
    y_train = np.array([i for i in range(109) for j in range(5)])
    y_test = np.array([i for i in range(109)])
    remaining_chans = list(range(64))

    print('Start -> ', end=' ')
    accuracy_scores[0] = cross_validate(Mahalanobis(), remaining_chans, protocol)
    print(64, end=' ')

    for it in range(62):
        acc_tmp = [0]*len(remaining_chans)
        for e, chan in enumerate(remaining_chans):
            chan_set = remaining_chans.copy()
            chan_set.remove(chan)
            acc_tmp[e] = cross_validate(Mahalanobis(), chan_set, protocol)
        max_idx = np.argmax(acc_tmp)
        ranked_chans[protocol][it] = remaining_chans[max_idx]
        accuracy_scores[it+1] = acc_tmp[max_idx]
        remaining_chans.pop(max_idx)
        print(63-it, end=' ')

    acc_tmp = [0]*2
    for e, chan in enumerate(remaining_chans):
        chan_set = remaining_chans.copy()
        chan_set.remove(chan)
        acc_tmp[e] = cross_validate(NearestCentroid('euclidean'), chan_set, protocol) # mahalanobis doesn't work for datasets with 1 feature
    max_idx = np.argmax(acc_tmp)
    ranked_chans[protocol][-2] = remaining_chans[max_idx]
    accuracy_scores[-1] = acc_tmp[max_idx]
    remaining_chans.pop(max_idx)
    print(1)
    ranked_chans[protocol][-1] = remaining_chans[0]
    print('\n\nDone!')

In [ ]:
# Fig. 6a

plt.figure(figsize=(10, 10))
plt.plot(ranked_chans['EO'][1], 'r')
plt.xticks(ticks=np.arange(4, 55, 10), labels=np.arange(60, 9, -10), fontsize=14)
plt.gca().invert_xaxis()
plt.ylim([10, 105])
plt.yticks(fontsize=14)
plt.plot(ranked_chans['EC'][1], 'b')
plt.xlabel('Remaining Channels', fontsize=18), plt.ylabel('Accuracy (%)', fontsize=18)
plt.legend(['Eyes Open', 'Eyes Close'], loc='lower right', fontsize=16)
plt.scatter(44, (ranked_chans['EO'][1][44]+ranked_chans['EC'][1][44])/2, 300, c='k', marker='|')
plt.plot([44, 44], [ranked_chans['EC'][1][44], 0], '--k')
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
# plt.savefig('RFE.png', dpi=200, bbox_inches='tight')

In [ ]:
print('EO:', np.argmax(np.flip(ranked_chans['EO'][1]))+1, 'channels', np.max(ranked_chans['EO'][1]).round(1), '% accuracy')
print('EC:', np.argmax(np.flip(ranked_chans['EC'][1]))+1, 'channels', np.max(ranked_chans['EC'][1]).round(1), '% accuracy')

In [ ]:
# Fig. 6b

# topography visualized in matlab
channel_ranks = {'EO': np.argsort(ranked_chans['EO'][0]), 'EC': np.argsort(ranked_chans['EC'][0])}

In [ ]:
savemat('channel_ranks.mat', channel_ranks)

# Covariance Matrix Analysis

In [ ]:
# Table 3

seed(0)
subset_size = 80
protocols = ['EO', 'EC']
accuracy_scores_loc = {'EO': [], 'EC': []}
accuracy_scores_uni = {'EO': [], 'EC': []}
accuracy_scores_non = {'EO': [], 'EC': []}
y_train = np.array([i for i in range(subset_size) for j in range(5)])
y_test = np.array([i for i in range(subset_size)])
for protocol in protocols:
    for _ in range(50):
        start = randint(0, 109-subset_size)
        subjects = slice(start, start+subset_size)
        othersubjects = list(range(start))+list(range(start+subset_size, 109))
        covoth = np.cov(mcl[protocol][othersubjects, :, :, :].reshape(-1, 64).transpose())
        accuracy_scores_loc_ = []
        accuracy_scores_uni_ = []
        accuracy_scores_non_ = []
        for i in range(6):

            train_set = list(range(6))
            test_set = [train_set.pop(i)]

            x_train = mcl[protocol][subjects, train_set, :, :].reshape(-1, 64)
            x_test = mcl[protocol][subjects, test_set, :, :].reshape(-1, 64)
            
            # local covariance matrix
            model = Mahalanobis()
            model.fit(x_train, y_train)
            accuracy_scores_loc_.append(accuracy_score(y_test, model.predict(x_test))*100)
            
            # non-local covariance matrix
            model = Mahalanobis(VI=np.linalg.inv(covoth))
            model.fit(x_train, y_train)
            accuracy_scores_uni_.append(accuracy_score(y_test, model.predict(x_test))*100)
            
            # no covariance matrix
            model = Euclidean()
            model.fit(x_train, y_train)
            accuracy_scores_non_.append(accuracy_score(y_test, model.predict(x_test))*100)
        
        accuracy_scores_loc[protocol].append((np.mean(accuracy_scores_loc_), np.std(accuracy_scores_loc_)))
        accuracy_scores_uni[protocol].append((np.mean(accuracy_scores_uni_), np.std(accuracy_scores_uni_)))
        accuracy_scores_non[protocol].append((np.mean(accuracy_scores_non_), np.std(accuracy_scores_non_)))

mean_std = [np.mean(accuracy_scores_loc['EO'], 0).round(1), np.mean(accuracy_scores_loc['EC'], 0).round(1)]
print('Performance using local covariance matrix:\n',
      'EO: {} \N{PLUS-MINUS SIGN} {} %\t'.format(mean_std[0][0], mean_std[0][1]),
      'EC: {} \N{PLUS-MINUS SIGN} {} %\t'.format(mean_std[0][0], mean_std[0][1]))
mean_std = [np.mean(accuracy_scores_uni['EO'], 0).round(1), np.mean(accuracy_scores_uni['EC'], 0).round(1)]
print('Performance using non-local covariance matrix:\n',
      'EO: {} \N{PLUS-MINUS SIGN} {} %\t'.format(mean_std[0][0], mean_std[0][1]),
      'EC: {} \N{PLUS-MINUS SIGN} {} %\t'.format(mean_std[0][0], mean_std[0][1]))
mean_std = [np.mean(accuracy_scores_non['EO'], 0).round(1), np.mean(accuracy_scores_non['EC'], 0).round(1)]
print('Performance using no covariance matrix:\n',
      'EO: {} \N{PLUS-MINUS SIGN} {} %\t'.format(mean_std[0][0], mean_std[0][1]),
      'EC: {} \N{PLUS-MINUS SIGN} {} %\t'.format(mean_std[0][0], mean_std[0][1]))